In [ ]:
%pip install langchain
%pip install langchain-openai
%pip install langchain-pinecone
%pip install langchain-weaviate
%pip install langchain-community 
%pip install faiss-cpu 
%pip install tiktoken
%pip install openai
%pip install PyPDF2
%pip install faiss-cpu
%pip install python-dotenv


In [4]:
from PyPDF2 import PdfReader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain_pinecone import PineconeVectorStore
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

In [6]:
import os
load_dotenv()

True

In [7]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [9]:
reader = PdfReader('./data/data.pdf')

In [10]:
# read data from the file and put them into a variable named raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
       raw_text += text

In [11]:
raw_text

'•python3 -m venv .venv•source .venv/bin/activate•deactivate\n'

In [23]:
# split the text into smaller chunks to prevent hitting token limits during retrieval 
text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
)

texts = text_splitter.split_text(raw_text)

In [24]:
len(texts)

1

In [14]:
# see text in first chunk
texts[0]

'•python3 -m venv .venv•source .venv/bin/activate•deactivate'

In [ ]:
# see text in second chunk, this should start from charactor 800 in the first chunk based on the chunk_overlap setting defined within the CharatorTextSplitter
texts[1]

In [15]:
# download embeddings from OpenAI
embeddings = OpenAIEmbeddings()


In [16]:
# find the embeddings of the text chunks and store them in the docsearch variable
docsearch = FAISS.from_texts(texts, embeddings)

In [17]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [18]:
# create a chain
chain = load_qa_chain(OpenAI(), chain_type='stuff')

In [19]:
# create a query
query = "How do i create a virtual environment?"
docs = docsearch.similarity_search(query)
result = chain.invoke({
    "input_documents": docs,
    "question": query
})

In [20]:
result

{'input_documents': [Document(page_content='•python3 -m venv .venv•source .venv/bin/activate•deactivate')],
 'question': 'How do i create a virtual environment?',
 'output_text': ' To create a virtual environment, use the command "python3 -m venv .venv" and then activate it using "source .venv/bin/activate". To exit the virtual environment, use the command "deactivate".'}